In [1]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, Polygon
import folium
from random import *
from tqdm import tqdm

tqdm.pandas()

In [2]:
df_osrm = gpd.read_file('data/jkt_OSRM.geojson')
df_gmaps = gpd.read_file('data/jkt_GMAPS.geojson')
df_osmnx = gpd.read_file('data/jkt_OSMNX.geojson')
df = pd.read_feather('data/matched_jkt.ftr')

In [3]:
df_osrm

,trj_id,routes,osrm_dur,osrm_dist,geometry
0,67224,"{'route': [[-6.14041, 106.81426], [-6.13864, 1...",450.8,7994.5,"LINESTRING (106.81426 -6.14041, 106.81373 -6.1..."
1,66765,"{'route': [[-6.16999, 106.81102], [-6.17091, 1...",87.8,1068.3,"LINESTRING (106.81102 -6.16999, 106.81111 -6.1..."
2,54853,"{'route': [[-6.17941, 106.81706], [-6.17942, 1...",6.0,27.0,"LINESTRING (106.81706 -6.17941, 106.81681 -6.1..."
3,28910,"{'route': [[-6.25575, 106.86347], [-6.25575, 1...",53.5,770.5,"LINESTRING (106.86347 -6.25575, 106.86351 -6.2..."
4,67413,"{'route': [[-6.12432, 106.8714], [-6.12221, 10...",613.2,13962.8,"LINESTRING (106.87140 -6.12432, 106.86215 -6.1..."
...,...,...,...,...,...
892,41343,"{'route': [[-6.17404, 106.89217], [-6.17617, 1...",422.8,8432.2,"LINESTRING (106.89217 -6.17404, 106.89449 -6.1..."
893,46578,"{'route': [[-6.18097, 106.81731], [-6.18163, 1...",156.3,2103.6,"LINESTRING (106.81731 -6.18097, 106.81696 -6.1..."
894,68027,"{'route': [[-6.1867, 106.82318], [-6.19376, 10...",158.7,2810.2,"LINESTRING (106.82318 -6.18670, 106.82311 -6.1..."
895,66096,"{'route': [[-6.20055, 106.82328], [-6.20116, 1...",90.8,1538.4,"LINESTRING (106.82328 -6.20055, 106.82322 -6.2..."


In [4]:
df_gmaps

,trj_id,gdis,gdur,gdur_traffic,geometry
0,67224,7989,1055,1763,"LINESTRING (106.81424 -6.14042, 106.81396 -6.1..."
1,66765,1069,172,196,"LINESTRING (106.81102 -6.16999, 106.81110 -6.1..."
2,54853,1683,373,379,"LINESTRING (106.81706 -6.17943, 106.81803 -6.1..."
3,28910,764,110,243,"LINESTRING (106.86349 -6.25572, 106.86352 -6.2..."
4,67413,14528,1007,1787,"LINESTRING (106.87140 -6.12430, 106.86738 -6.1..."
...,...,...,...,...,...
892,41343,7285,918,1102,"LINESTRING (106.89216 -6.17406, 106.89259 -6.1..."
893,46578,2018,309,466,"LINESTRING (106.81731 -6.18097, 106.81702 -6.1..."
894,68027,2813,297,614,"LINESTRING (106.82319 -6.18670, 106.82315 -6.1..."
895,66096,1538,139,365,"LINESTRING (106.82328 -6.20055, 106.82322 -6.2..."


In [5]:
df_osmnx

,trj_id,osmnx_length,geometry
0,67224,8445.874560,"LINESTRING (106.81433 -6.14068, 106.81417 -6.1..."
1,66765,1299.788791,"LINESTRING (106.81101 -6.16979, 106.81106 -6.1..."
2,54853,85.565615,"LINESTRING (106.81655 -6.17942, 106.81655 -6.1..."
3,28910,253.062837,"LINESTRING (106.86351 -6.25575, 106.86351 -6.2..."
4,67413,380.214596,"LINESTRING (106.87213 -6.12395, 106.87263 -6.1..."
...,...,...,...
892,41343,8962.950452,"LINESTRING (106.89214 -6.17434, 106.89159 -6.1..."
893,46578,3781.024303,"LINESTRING (106.81739 -6.18083, 106.81717 -6.1..."
894,68027,3274.451970,"LINESTRING (106.82317 -6.18691, 106.82309 -6.1..."
895,66096,2684.401888,"LINESTRING (106.82339 -6.20111, 106.82337 -6.2..."


In [6]:
df

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,time,day,month,year,day_of_week,location
0,10186,motorcycle,ios,1555581210,-6.170755,106.787247,0.0,1,5.0,2019-04-18 16:53:30+00:00,18,4,2019,3,"[106.787248, -6.170731]"
1,10186,motorcycle,ios,1555581211,-6.169763,106.787521,0.0,3,5.0,2019-04-18 16:53:31+00:00,18,4,2019,3,"[106.787423, -6.170158]"
2,10186,motorcycle,ios,1555581212,-6.169143,106.787598,0.0,344,5.0,2019-04-18 16:53:32+00:00,18,4,2019,3,"[106.787683, -6.169139]"
3,10186,motorcycle,ios,1555581213,-6.169040,106.787605,0.0,335,5.0,2019-04-18 16:53:33+00:00,18,4,2019,3,"[106.787678, -6.169037]"
4,10186,motorcycle,ios,1555581214,-6.168501,106.787651,0.0,340,5.0,2019-04-18 16:53:34+00:00,18,4,2019,3,"[106.787662, -6.168501]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121998,999,car,ios,1554689981,-6.215565,106.817566,0.0,224,10.0,2019-04-08 09:19:41+00:00,8,4,2019,0,"[106.817528, -6.215525]"
121999,999,car,ios,1554689982,-6.215604,106.817520,0.0,225,10.0,2019-04-08 09:19:42+00:00,8,4,2019,0,"[106.817484, -6.215567]"
122000,999,car,ios,1554689983,-6.215643,106.817474,0.0,225,10.0,2019-04-08 09:19:43+00:00,8,4,2019,0,"[106.817441, -6.215608]"
122001,999,car,ios,1554689984,-6.215685,106.817421,0.0,226,10.0,2019-04-08 09:19:44+00:00,8,4,2019,0,"[106.817392, -6.215655]"


In [7]:
min_indices = []
max_indices = []
grouped = df.groupby("trj_id")
for name, group in tqdm(grouped):
    if not group.empty:
        min_indices.append(group.iloc[0].name)
        max_indices.append(group.iloc[-1].name)
    
df_start = df[df.index.isin(set(min_indices))].reset_index(drop=True)
df_end = df[df.index.isin(set(max_indices))].reset_index(drop=True)

100%|██████████| 55995/55995 [00:01<00:00, 48720.08it/s]


In [8]:
df_start

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,time,day,month,year,day_of_week,location
0,10186,motorcycle,ios,1555581210,-6.170755,106.787247,0.00,1,5.0,2019-04-18 16:53:30+00:00,18,4,2019,3,"[106.787248, -6.170731]"
1,10255,car,ios,1554821644,-6.190298,106.762566,0.00,87,4.0,2019-04-09 21:54:04+00:00,9,4,2019,1,"[106.762568, -6.190339]"
2,10286,motorcycle,ios,1554979664,-6.151320,106.815994,0.00,359,5.0,2019-04-11 17:47:44+00:00,11,4,2019,3,"[106.815996, -6.15126]"
3,10620,car,ios,1555317881,-6.167817,106.787636,6.16,66,10.0,2019-04-15 15:44:41+00:00,15,4,2019,0,"[106.787655, -6.167817]"
4,10995,car,ios,1555169718,-6.143500,106.787987,0.00,260,5.0,2019-04-13 22:35:18+00:00,13,4,2019,5,"[106.787983, -6.143472]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,9173,motorcycle,ios,1555414149,-6.210269,106.817917,0.00,153,10.0,2019-04-16 18:29:09+00:00,16,4,2019,1,"[106.817894, -6.210269]"
198,9245,motorcycle,ios,1554851548,-6.219659,106.813873,0.00,170,10.0,2019-04-10 06:12:28+00:00,10,4,2019,2,"[106.813898, -6.219666]"
199,9588,car,ios,1554789840,-6.168535,106.788383,0.00,165,10.0,2019-04-09 13:04:00+00:00,9,4,2019,1,"[106.788467, -6.168548]"
200,9636,car,ios,1555670638,-6.142925,106.787964,0.00,183,10.0,2019-04-19 17:43:58+00:00,19,4,2019,4,"[106.787964, -6.142925]"


In [9]:
df_end

,trj_id,driving_mode,osname,pingtimestamp,rawlat,rawlng,speed,bearing,accuracy,time,day,month,year,day_of_week,location
0,10186,motorcycle,ios,1555582576,-6.203257,106.823425,0.00,97,10.0,2019-04-18 17:16:16+00:00,18,4,2019,3,"[106.823441, -6.203175]"
1,10255,car,ios,1554822109,-6.189587,106.795097,0.00,88,8.0,2019-04-09 22:01:49+00:00,9,4,2019,1,"[106.795096, -6.189563]"
2,10286,motorcycle,ios,1554980652,-6.125361,106.798851,0.00,189,5.0,2019-04-11 18:04:12+00:00,11,4,2019,3,"[106.798851, -6.125346]"
3,10620,car,ios,1555317740,-6.173262,106.789268,5.39,33,5.0,2019-04-15 15:42:20+00:00,15,4,2019,0,"[0.0, 0.0]"
4,10995,car,ios,1555170303,-6.166469,106.786736,0.00,141,5.0,2019-04-13 22:45:03+00:00,13,4,2019,5,"[106.786739, -6.166447]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,9173,motorcycle,ios,1555415020,-6.166227,106.802322,0.00,351,5.0,2019-04-16 18:43:40+00:00,16,4,2019,1,"[106.802269, -6.166576]"
198,9245,motorcycle,ios,1554852374,-6.265384,106.783623,0.00,151,10.0,2019-04-10 06:26:14+00:00,10,4,2019,2,"[106.78369, -6.265383]"
199,9588,car,ios,1554790684,-6.242610,106.842880,0.00,105,10.0,2019-04-09 13:18:04+00:00,9,4,2019,1,"[106.84288, -6.242602]"
200,9636,car,ios,1555671587,-6.176974,106.793724,0.00,308,10.0,2019-04-19 17:59:47+00:00,19,4,2019,4,"[106.793697, -6.177003]"


In [10]:
# Adding lines to the different feature groups
for sample_trj_id in set(df.trj_id):
    jkt = folium.Map(location=list(map(lambda x: x[::-1], df_osrm[df_osrm.trj_id == str(sample_trj_id)].geometry.iloc[0].coords))[0], zoom_start=17, tiles='openstreetmap')

    # Creating feature groups
    f1 = folium.FeatureGroup("OSRM")
    f2 = folium.FeatureGroup("GMAPS")
    f3 = folium.FeatureGroup("OSMNX")
    f4 = folium.FeatureGroup("ACTUAL")
    
    temp = df[df.trj_id == str(sample_trj_id)]
    folium.vector_layers.PolyLine(list(map(lambda x: x[::-1], df_osrm[df_osrm.trj_id == str(sample_trj_id)].geometry.iloc[0].coords)), popup='<b>OSRM</b>', color='red', weight=3).add_to(f1)
    folium.vector_layers.PolyLine(list(map(lambda x: x[::-1], df_gmaps[df_gmaps.trj_id == str(sample_trj_id)].geometry.iloc[0].coords)), popup='<b>GMAPS</b>', color='green', weight=3).add_to(f2)
    folium.vector_layers.PolyLine(list(map(lambda x: x[::-1], df_osmnx[df_osmnx.trj_id == str(sample_trj_id)].geometry.iloc[0].coords)), popup='<b>OSMNX</b>', color='blue', weight=3).add_to(f3)
    folium.vector_layers.PolyLine(list(zip(list(temp['rawlat']), list(temp['rawlng']))), popup='<b>ACTUAL</b>', color='black', weight=5).add_to(f4)

    f1.add_to(jkt)
    f2.add_to(jkt)
    f3.add_to(jkt)
    f4.add_to(jkt)
    folium.LayerControl().add_to(jkt)
    jkt.save(f'data/routes/matched/{sample_trj_id}.html')
jkt # sample

In [11]:
df_merged = df_gmaps[df_gmaps.trj_id.isin(df.trj_id)].copy()
df_merged.rename(columns={'geometry':'geometry_gmaps'}, inplace=True)
df_merged['osrmdur'] = df_osrm['osrm_dur']
df_merged['osrmdis'] = df_osrm['osrm_dist']
df_merged['osmnxdis'] = df_osmnx['osmnx_length']
df_merged = df_merged.sort_values(by='trj_id').reset_index(drop=True)
df_merged['starth'] = df_start['time'].apply(lambda x: x.hour)
df_merged['endh'] = df_end['time'].apply(lambda x: x.hour)
df_merged['geometry_osrm'] = df_osrm['geometry']
df_merged['geometry_osmnx'] = df_osmnx['geometry']
df_merged = df_merged[['trj_id', 'osmnxdis', 'osrmdis', 'gdis', 'osrmdur', 'gdur', 'gdur_traffic', 'starth', 'endh', 'geometry_osmnx', 'geometry_osrm', 'geometry_gmaps']]
df_merged.head()

,trj_id,osmnxdis,osrmdis,gdis,osrmdur,gdur,gdur_traffic,starth,endh,geometry_osmnx,geometry_osrm,geometry_gmaps
0,10186,16967.311474,10433.0,7849,603.6,979,1366,16,17,"LINESTRING (106.81433 -6.14068, 106.81417 -6.1...","LINESTRING (106.81426 -6.14041, 106.81373 -6.1...","LINESTRING (106.78732 -6.17077, 106.78751 -6.1..."
1,10255,5392.254356,4701.1,4044,302.6,448,609,21,22,"LINESTRING (106.81101 -6.16979, 106.81106 -6.1...","LINESTRING (106.81102 -6.16999, 106.81111 -6.1...","LINESTRING (106.76257 -6.19032, 106.76537 -6.1..."
2,10286,7660.991620,4504.2,4558,370.9,677,925,17,18,"LINESTRING (106.81655 -6.17942, 106.81655 -6.1...","LINESTRING (106.81706 -6.17941, 106.81681 -6.1...","LINESTRING (106.81600 -6.15129, 106.81701 -6.1..."
3,10620,5171.606425,1610.8,2132,233.2,415,554,15,15,"LINESTRING (106.86351 -6.25575, 106.86351 -6.2...","LINESTRING (106.86347 -6.25575, 106.86351 -6.2...","LINESTRING (106.78765 -6.16782, 106.78764 -6.1..."
4,10995,4850.560287,5005.2,5420,263.9,641,790,22,22,"LINESTRING (106.87213 -6.12395, 106.87263 -6.1...","LINESTRING (106.87140 -6.12432, 106.86215 -6.1...","LINESTRING (106.78799 -6.14350, 106.78688 -6.1..."


In [12]:
df_merged.to_csv('data/merged_routes.csv', index=False)

In [13]:
sum(df_merged['osrmdis'] > df_merged['gdis']), sum(df_merged['osrmdur'] > df_merged['gdur'])

(107, 2)

In [14]:
df_osrm[df_osrm.trj_id == '8349'].routes.iloc[0]['route']
# http://router.project-osrm.org/route/v1/driving/106.91498,-6.14524;106.90961,-6.14821

[[-6.14524, 106.91498],
 [-6.1464, 106.91535],
 [-6.14811, 106.91568],
 [-6.14866, 106.91573],
 [-6.14868, 106.91474],
 [-6.14835, 106.9096],
 [-6.1483, 106.90941],
 [-6.1481, 106.90943],
 [-6.14815, 106.90962],
 [-6.14821, 106.90961]]